# Feature Selection

multicollinearity <br>

# Base Models

# Feature Selection

multicollinearity <br>
base logistic model and look at weights of paramaters <br>
z test for all engineered features

# Improved Models

# Final Model

Now that we've made our insights from these values, we'll remove the original columns and keep their logged counterparts, as mentioned in our Data Cleaning section.

In [ ]:
logged_columns = [x for x in list(df.columns) if "LOG_" in x]
dropping_columns = [x.replace("LOG_","") for x in logged_columns]
df.drop(columns = dropping_columns,inplace=True)